In [1]:
# add autocbh package to path
import sys
sys.path.append('../autocbh')

# import relevant rdkit 
from rdkit import Chem
# drawing tools used for CBH.buildCBH visualization
from rdkit.Chem import Draw, rdChemReactions
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
from IPython.display import SVG, display

# Import CBH generator
from CBH import buildCBH
from calcCBH import calcCBH
from TN import thermochemical_network, visualize
from UQ import uncertainty_quantification

# only needed for np.inf example for now
import numpy as np
from numpy import inf
import pandas as pd

In [2]:
# Initialize module
c = calcCBH(methods=[], # CHANGED
            dataframe_path='../data/pfas_energies.pkl', # supply pickled dataframe 
            method_keys_path='../data/methods_keys.yaml', # supply yaml file mapping method names to each QM energy column name
            rankings_path='../data/rankings.yaml', # supply yaml file mapping each method to a level of theory ranking
            zero_out_heats=True) # ensures that the heats of formation/rxn for nonexperimental values are converted to 0 from nan


In [3]:
c.calc_Hf(saturate=[1, 9], # only use CBH-H
          max_rung= None, # can choose a maximum rung, otherwise will do highest possible
          alt_rxn_option=None, 
          priority='rel_coeff',
          surface_smiles=None)

Process completed with errors in 22 species
These errors are likely to have propagated for the calculation of heats of formation of larger species.
Updating the reference values of species in the database will improve accuracy of heats of formation.
To inspect the errors, run the calcCBH.print_errors() method.


,DfH,DrxnH,source
smiles,,,
F,-272.680000,0.000000,ATcT
O,-238.898000,0.000000,ATcT
[H][H],0.000000,0.000000,ATcT
C,-66.551000,0.000000,ATcT
CF,-227.480000,0.000000,ATcT
...,...,...,...
FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F,-3382.594855,2.032257,"CBHavg-(4-H, 3-F)//m062x_dlpno+wb97xd_dlpno"
CC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(C)(F)F,-2615.259675,2.145206,"CBHavg-(4-H, 6-F)//m062x_dlpno+wb97xd_dlpno"
CC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F,-3205.459270,0.400158,"CBHavg-(4-H, 6-F)//m062x_dlpno"


In [5]:
smiles = 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F'
c.error_messages[smiles] = []
label = 'H'
cbh = buildCBH(smiles, label)

coeffs =[]
rungs = list(reversed(range(6+1)))
for rung in rungs:
    c._check_rung_usability(smiles, 
                            test_rung=rung,
                            cbh_rcts=cbh.cbh_rcts,
                            cbh_pdts=cbh.cbh_pdts,
                            label=label)
    print(c.rxns[smiles][label])
    coeffs.append(sum(abs(np.array(list(c.rxns[smiles][label].values())))))
    print(coeffs[-1])
if min(coeffs) == coeffs[0]:
    print(rungs[np.where(coeffs == coeffs[0])[0][-1]])
else:
    print(rungs[np.where(coeffs < coeffs[0])[0][0]])

{'CC(F)(F)C(F)(F)C(=O)O': 1, 'CC(F)(F)C(F)(F)C(C)(F)F': 3, 'CC(F)(F)C(F)(F)C(F)(F)F': 1, 'CC(F)(F)C(C)(F)F': -4}
9
{'CC(F)(F)C(F)(F)C(=O)O': 1, 'CC(F)(F)C(F)(F)C(C)(F)F': 3, 'CC(F)(F)C(F)(F)C(F)(F)F': 1, 'CC(F)(F)C(C)(F)F': -4}
9
{'CC(F)(F)C(F)(F)C(=O)O': 1, 'CC(F)(F)C(F)(F)C(C)(F)F': 3, 'CC(F)(F)C(F)(F)C(F)(F)F': 1, 'CC(F)(F)C(C)(F)F': -4}
9
{'CC(F)(F)C(F)(F)F': 1, 'CC(F)(F)C(=O)O': 1, 'CC(F)(F)C(C)(F)F': 4, 'CC(C)(F)F': -5}
11
{'CC(=O)O': 1, 'CC(C)(F)F': 5, 'CC(F)(F)F': 1, 'CC': -6}
13
{'CC': 6, 'C=O': 1, 'CO': 1, 'CF': 13, 'C': -20}
41
{'O': 2, 'F': 13, 'C': 7, '[H][H]': -22.0}
44.0
4


In [4]:
smiles = 'O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F'
c.error_messages[smiles] = []
label = 'F'
cbh = buildCBH(smiles, label)
c._decompose_rxn(smiles, 
                 test_rung=6,
                 cbh_rcts=cbh.cbh_rcts,
                 cbh_pdts=cbh.cbh_pdts,
                 label=label)

([{'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)F': 4,
   'FC(F)(F)C(F)(F)C(F)(F)F': -4,
   'O=C(O)C(F)(F)C(F)(F)F': 1},
  {'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)F': 4,
   'O=C(O)C(F)(F)C(F)(F)F': 1,
   'FC(F)(F)C(F)(F)C(F)(F)F': -4},
  {'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)F': 4,
   'O=C(O)C(F)(F)C(F)(F)F': 1,
   'FC(F)(F)C(F)(F)C(F)(F)F': -4},
  {'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)F': 4,
   'O=C(O)C(F)(F)C(F)(F)F': 1,
   'FC(F)(F)C(F)(F)C(F)(F)F': -4},
  {'FC(F)(F)C(F)(F)C(F)(F)F': 5, 'O=C(O)C(F)(F)F': 1, 'FC(F)(F)C(F)(F)F': -5},
  {'FC(F)(F)C(F)(F)F': 6, 'OC(F)(F)F': 1, 'O=C(F)F': 1, 'FC(F)(F)F': -7},
  {'C': 3.75, 'O': 2, 'FC(F)(F)F': 3.25, '[H][H]': -9.0}],
 3)

In [6]:
c.rxns[smiles][label]

{'CC(F)(F)C(F)(F)C(C)(F)F': 3,
 'CC(F)(F)C(F)(F)C(F)(F)F': 1,
 'CC(F)(F)C(C)(F)F': -4,
 'CC(F)(F)C(F)(F)C(=O)O': 1}

In [ ]:
c.rxns['O=C(O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F']